<a href="https://colab.research.google.com/github/iPoetDev/ibm-skills-ai-colab-sessions/blob/main/notebooks-labs/Session3_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <ins>Session 3</ins>.1: **IBM Skills Build: Generative AI Live Technical Lab** (Part 1)

> #### **Objective**: *Understand the theory and hands-on implementation of*: <br>  1️⃣ Variational AutoEncoders(VAE)
>> - Exploring AutoEncoders 4 layers
>> - Displaying generated images, like handwriting

- **URL**: [https://skills.yourlearning.ibm.com/activity/PLAN-CB1CC0D21AFB](https://skills.yourlearning.ibm.com/activity/PLAN-CB1CC0D21AFB "Programme for Artifical Intelligence: eLearning on IBM.com (Login required)") &nbsp;<small><sup><strong> eLearning, Login</strong></sup></small><br>
- **Share**: [Introduction to Generative AI](https://skills.yourlearning.ibm.com/activity/MDL-388 "eLearning on IBM.com (Login required") &nbsp;<small><sup><strong>eLearning, Login</strong></sup></small>
- **Recording**: [Recording: Live Technical Session 3](https://skills.yourlearning.ibm.com/activity/URL-6BF19B3CC379 "Video: IBM's Box.com (Login required")
- **CoLab: Source Notebook**: [https://colab.research.google.com/drive/1eD7pRKmhVFl0nfwzsoIy9RTtoPMVkZPW?usp=sharing](https://colab.research.google.com/drive/1eD7pRKmhVFl0nfwzsoIy9RTtoPMVkZPW?usp=sharing "Authors: Marty Bradly's Session 3 VAE notebook")
  - Original by author: Marty Bradly: [LinkedIn](https://www.linkedin.com/in/martybradley/), [Website](https://www.evergreen-ai.com/), [GitHub @marty916](https://github.com/marty916 "Marty Bradly [July, 2024], Last accessed: July 2024")

<small>Notebook for technical audiences | See README and Sessions.md for business and product audiences</small>

> <hr>

## GitHub

- **IBM-Skills-AI_Colab-Sessions**:
    - [README](https://github.com/iPoetDev/ibm-skills-ai-colab-sessions/blob/main/README.md)
    - [Sessions Summary](https://github.com/iPoetDev/ibm-skills-ai-colab-sessions/blob/main/Sessions.md)
    - [notebook-labs/Session3_VAE.ipynb](https://github.com/iPoetDev/ibm-skills-ai-colab-sessions/blob/main/notebooks-labs/Session3_VAE.ipynb "@iPoetDev: GitHub.com:  IBM-Skills-AI_Colab-Sessions: Session3_VAE Juypter Notebook")

## Steps

1.   [Setup/Imports](#scrollTo=UGD_IeVp_zpv&line=1&uniqifier=1)
2.   [Load dataset](#scrollTo=Ptv08Kdvfn9I&line=1&uniqifier=1)
3.   [Encoder](#scrollTo=aZZPo9awf2RE&line=1&uniqifier=1)
4.   [VAE Sampling](#scrollTo=zLH8KVSff8pH&line=1&uniqifier=1)
5.   [Decoder](#scrollTo=9nrEB_ITgC0I&line=1&uniqifier=1)
6.   [VAE Model](#scrollTo=rjShiZR3gGLq&line=1&uniqifier=1)
7.   [VAE Loss](#scrollTo=iZPZR3HngJp8&line=1&uniqifier=1)
8.   [Model Training](#scrollTo=Y4SeQikcgM8i&line=1&uniqifier=1)
9.   [Display Function: Plot](#scrollTo=zho9LQCegQe0&line=1&uniqifier=1)
10.  [Model Execution](#scrollTo=RFOnrVypgWu6&line=2&uniqifier=1)



---
> <hr>
---

## 1. <ins> Setup / Imports<ins>

*   TensorFlow
    - Shaping in Sampling
    - Keras (below)
*   Keras.io / TensorFlow Keras:
    - Datasets : MINST Data
    - Inputs: Encoders, Decoder's Latent representaton inputs
    - Models: VAE Model (Encoder, Decoder)
    - Losses: Binary (Cross Entropy)
    - Backend:
        - Random Noise to Sampling,
        - Calculations for VAE losses.
*   NumPy: Contain/Manipluate Data units.
*   MatplotLib: Ploting / generating graphics


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

> <hr>

## 2. <ins>Load the MNIST dataset</ins>

> Load MNIST dataset - pictures of handwritten numbers

1. Load Dataset
2. Conversion: Floats, Normalise per Train/Test sets
3. Reshape the pictures
   - By transforms the origina; data set array into a 4D array where each element represents a single grayscale image of size 28x28 pixels.
   -  This format is commonly used as input for CNNs

- Consants for cleaner code

In [ ]:
DATA_NORMALISE = 255
DATA_FLOAT = 'float32'
DATA_WIDTH = 28
DATA_HEIGHT = 28
DATA_CHANNELS = 1

In [ ]:

# Load MNIST dataset - pictures of handwritten numbers
(x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()

# Convert the Data to
# i) Floats
# ii) Normalize
# AS data is made with numbers between 0 and 255

# A) Training Set
x_train = x_train.astype(DATA_FLOAT) / DATA_NORMALISE.

# B) Test Set
x_test = x_test.astype(DATA_FLOAT) / DATA_NORMALISE.

## ===

# Reshapping: the pictures for computer so that each picture is
# - 28x28
# - 1 color channel
# Pictures: 28x28 pixels, each pixel assigned a number to shows how dark it is.
# - as pictures because they are black and white pictures (1 channel)

# Reshape: Train Set

# np.reshape: changes array shape
x_train = np.reshape(x_train, (len(x_train),  # len() Nos of units in set
                               DATA_WIDTH,    # width of each unit
                               DATA_HEIGHT,   # height of each unit
                               DATA_CHANNELS))
                                    # number of channels, greyscale

# Reshape: Test Set
x_test = np.reshape(x_test, (len(x_test),
                               DATA_WIDTH,
                               DATA_HEIGHT,
                               DATA_CHANNELS))

> <hr>

## 3. <ins>Encoder</ins>

1.   Set Encoder's Latent Dim
2.   Setup Encoder's Input Shape
3.   Define 1st Convoluntional Layer
     - Applies filter to the input image
       - Used to highlight important features
       - Use 32 filters
       - Per filter size: 3x3 pixels
     - Use `relu` activation
       - choice of activation function depends on the specific task
       - the desired properties of the network.
     - Use Strides to +2px/per move
     - Padding: Output size === Input size
4.  Flatten to Single Dimension
    - the output of convolutional layers (CNN) is typically a multi-dimensional tensor.
       -  a multi-dimensional tensor - represent feature maps.
    - transforms this multi-dimensional tensor into a 1D vector.
5.  Neuornal Density
    - Is a fully connected (dense) layer
    - Each neuron in this layer is connected to every neuron in the previous layer.
    - Performs a linear transformation on the flattened input x
    - Applies an activation function


- Why Flatten? Flattening is often necessary before connecting to a fully connected (dense) layer, as dense layers expect their input to be a 1D vector
- Combining Flattening with neuronal density, is common in  neural networks, especially after convolutional layers, to enable learning complex patterns and relationships in the data.


6.  Latent Space Characteristics
    - Crucial components of VAE architecture<br>
    a. Mean of Latent Space<br>
    b. Log-Variance of Latent Space
    -  A lower-dimensional representation of the input data, capturing its essential features.
    - Mean and Variance: The mean and variance of the latent space distribution control the location and spread of the encoded data points in the latent space.

- Constants for clear code and annotation

In [ ]:
LATENT_DIM_SIZE = 2     # Dimensionality: Latent Space: Complexity of represent.
DATA_SHAPE = (DATA_WIDTH, DATA_HEIGHT, DATA_CHANNELS)
FILTER_SIZE = 3         # 3x3 Filter size
ACTIVATION_FUNCTION = 'relu' #  'relu', 'sigmoid', and 'tanh'
STRIDE = 2              # Nos of Pixels to move by
PADS = 'same'           # Padding: Output size === Input size
FEATURES_SIMPLE = 32    # 32 Filters for Important Features
FILTERS_COMPLEX = 64    # 64 Filters for more Complex Features
LAYER_NEURONS = 16      # Specifies nos of neurons (units) in the dense layer.

In [ ]:
# Encoder
latent_dim = LATENT_DIM_SIZE

# Setting up the input encoder, the shape must match our data
encoder_inputs = tf.keras.Input(shape=DATA_SHAPE)

## 2 Dimensions+

# First convolutional layer: Simple/Important Features (32)
x = layers.Conv2D(FEATURES_SIMPLE,
                  FILTER_SIZE,
                  activation=ACTIVATION_FUNCTION,
                  strides=STRIDE,
                  padding=PADS)(encoder_inputs)

# Similar Convolutional layer: Complex Features (64)
x = layers.Conv2D(FEATURES_COMPLEX,
                  FILTER_SIZE,
                  activation=ACTIVATION_FUNCTION,
                  strides=STRIDE,
                  padding=PADS)(x)

## 1 Dimensions

# Flattens to 1D
x = layers.Flatten()(x)
x = layers.Dense(LAYER_NEURONS,
                 activation=ACTIVATION_FUNCTION,)(x)

## Latent Space Characteristics
# Represents the latent space, basically summarizing in just a few key points

# creates a dense layers maping the input to a vector:
# 1) mean of the latent space distribution.
z_mean = layers.Dense(latent_dim)(x)
# 2) logarithm of the variance of the latent space distribution.
z_log_var = layers.Dense(latent_dim)(x)



> <hr>

## 4. <ins>VAE Sampling</ins>

> This function is generating new samples in the latent space by adding
some random noise to the simplified data representation. <br>
> This is important for creating diverse and realistice outputs in the models like a VAE.

1.   Sampling Function
     - Batching
     - Dimensionality
     - Randon Noise
     - Return New Sample
2.   Layers Lambda sampler


In [ ]:
# VAE Sampling Constants

BATCH_SCOPE = 0
DIM_SCOPE = 1
EXP_SCOPE = 0.5

In [ ]:
# Sampling function for the VAE

def sampling(args):
    # Split args
    z_mean, z_log_var = args
    # Sampling: Batch
    batch = tf.shape(z_mean)[BATCH_SCOPE]
    # Sampling: Dimension
    dim = tf.shape(z_mean)[DIM_SCOPE]
    # Sampling: Epsilon | Random Noise
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    # Sampling: Return New Sample
    return z_mean + tf.keras.backend.exp(EXP_SCOPE * z_log_var) * epsilon

# the function we just defined using a sampling function 'Lambda'
z = layers.Lambda(sampling)([z_mean,
                            z_log_var])



> <hr>

## 5. <ins>Decoder</ins>

> - This code is building the decoder part of a Variational Autoencoder (VAE).
  - The decoder takes the simplified latent representation (latent_dim) and transforms it back into the original image format through a series of layers.
> - These layers gradually upscale and reshape the data until it matches the size of the original input images, effectively reconstructing the images from the compressed latent space.

1.   Set Latent Dimensionality
2.   Decoder Inputs define
3.   Define Layer Density
4.   Reshape Layers
5.   Conv2DTranspose Layers:
     - Upsample and extract features
     - i. Most features
     - ii. Highlighted features
6.   Output Layer:
     - Generate the final reconstructed image:
     - Sigmoid activation.


In [ ]:
LATENT_DIM_SIZE = 2      # Dimensionality: Latent Space: Complexity of represent
SPATIAL_ALPHA = 7        # First Spatial Dimensions
SPATIAL_OMEGA = 7        # Second Spatial Dimensions
DECODE_KERNEL_SIZE = 3       # 3x3 Filter size, A SQUARE kernel.
DECODE_KERNEL_CPLX = 64      # 32 channel/filter/kernel
DECODE_KERNEL_SMPL = 32      # 32 channel/filter/kernel
DECODE_KERNEL_ONE = 1        # Single channel/filter/kernel
DENSE_LAYER_UNITS = SPATIAL_ALPHA * SPATIAL_OMEGA * DECODE_NEURONS_COMPLEX
ACTIVATION_FUNCTION = 'relu'   # 'relu', 'sigmoid', and 'tanh'
ACTIVATION_DECODE = 'sigmoid'  # 'relu', 'sigmoid', and 'tanh'
STRIDE = 2                   # Nos of Pixels to move by
PADS = 'same'                # Padding: Output size === Input size

In [ ]:
# Decoder

# Latent Space Dimensionality
latent_dim = LATENT_DIM_SIZE

# Decoder In
decoder_inputs = tf.keras.Input(shape=(latent_dim,))

# Decoder Layers
x = layers.Dense(DENSE_LAYER_UNITS,
                 activation=ACTIVATION_FUNCTION)(decoder_inputs)

# Reshape Layers Reshape to prepare for convolutional transpose layers
x = layers.Reshape((SPATIAL_ALPHA,
                    SPATIAL_OMEGA,
                    DECODE_NEURONS_COMPLEX))(x)

# Conv2DTranspose Layers: Upsample and extract features: More features
x = layers.Conv2DTranspose(DECODE_KERNEL_CPLX,
                           DECODE_KERNEL_SIZE,
                           activation=ACTIVATION_FUNCTION,
                           strides=STRIDE,
                           padding=PADS)(x)

# Conv2DTranspose Layers: Upsample / extract features: More highlighted features
x = layers.Conv2DTranspose(DECODE_KERNEL_SMPL,
                           DECODE_KERNEL_SIZE,
                           activation=ACTIVATION_FUNCTION,
                           strides=STRIDE,
                           padding=PADS)(x)

# Output Layer: Generate the final reconstructed image: sigmoid.
decoder_outputs = layers.Conv2DTranspose(DECODE_KERNEL_ONE,
                                         DECODE_KERNEL_SIZE,
                                         activation=ACTIVATION_DECODE,
                                         padding=PADS)(x)

> <hr>

## 6. <ins>VAE Model</ins>

> This code is creating a Variational Autoencoder (VAE) by
> - combining the encoder and decoder models.

1.   Encoder Model
     - compresses the input images into a simplified latent representation
2.   Decoder Model
     - reconstructs the images from this latent space.
4.   VAE Outputs - Final
     - Encoding:
       - input encoder_inputs is passed through the encoder model
       - compresses the input into a lower-dimensional latent representation.
     - Extracting Latent Representation
       - outputs a list containing the mean, log variance, and the sampled latent representation
       - extracts the sampled latent representation using index ENCODER_INDEX
           - assumed: the correct index for the latent representation
     - Decoding: reconstructs the original input from this compressed representation
     - Output: the final output of the VAE model.    
3.   VAE Model
     - final VAE model takes input images.
     - this processes them through the encoder to get the latent representation.
     - use the decoder to output the reconstructed images.


- Constants for cleaner code


In [ ]:
# VAE Model Constants

ENCODER_NAME = 'encoder'
DECODER_NAME = 'decoder'
VAE_NAME = 'vae'
ENCODER_INDEX = 2

In [ ]:
# VAE Model

# Encoding: Compressed Images
encoder = tf.keras.Model(encoder_inputs,
                         [z_mean,
                          z_log_var,
                          z],
                         name=ENCODER_NAME)

# Decoding: Reconstructs Images
decoder = tf.keras.Model(decoder_inputs,
                         decoder_outputs,
                         name=DECODER_NAME )

# Final VAE decodings by encoding inputs
outputs = decoder(encoder(encoder_inputs)[ENCODER_INDEX])

# VAE Model
vae = tf.keras.Model(encoder_inputs,
                     outputs,
                     name=VAE_NAME)

> <hr>

## 7. <ins>VAE Loss</ins>

> This code block is defining and adding a custom loss function to the Variational Autoencoder (VAE).

1. Custom Loss Function (VAE)
  - combines the reconstruction loss,
    - measures how well the VAE reconstructs the input images, and
  - KL divergence loss,
    - ensures the latent space is well-behaved and regularized.
  

2. The combined loss helps the VAE learn to generate realistic and diverse outputs.
3. Finally, the VAE is compiled with
the Adam optimizer for training

In [ ]:
DATA_WIDTH_LOSS = DATA_WIDTH
DATA_HEIGHT_LOSS = DATA_HEIGHT
KL_BELL_CURVE_CONSTANT
KL_AXIS = -1
KL_LOSS_SCALE = -0.5
WEIGHT_BIAS_OPTIMISER = 'adam'

In [ ]:
# VAE Loss

## Reconstruction Loss
# Binary Cross Entropy
reconstruction_loss = tf.keras.losses.binary_crossentropy(
                           tf.keras.backend.flatten(encoder_inputs),
                           tf.keras.backend.flatten(outputs)
                       )

# Scalling the Loss by input image dimensions
reconstruction_loss *= DATA_WIDTH_LOSS * DATA_HEIGHT_LOSS

## KL divergence loss
# KL Block 1:  Element-wise Calculation: Calculating KL divergence/element
kl_loss = KL_BELL_CURVE_CONSTANT
kl_loss += z_log_var
kl_loss -= tf.keras.backend.square(z_mean)
kl_loss -= tf.keras.backend.exp(z_log_var)

# KL Block 2: Aggregation & Scaling: aggregates these individual losses & scales them
kl_loss = tf.keras.backend.sum(kl_loss,
                               axis=KL_AXIS)
kl_loss *= KL_LOSS_SCALE

## VAE Loss
# Ave the sum of the reconstruction loss & KL divergence loss.
vae_loss = tf.keras.backend.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)

## Optimisation: Adam: Weights & Biases: Update models parameters, on calc loss.
vae.compile(optimizer=WEIGHT_BIAS_OPTIMISER)

> <hr>

# 8. <ins>Train the model</ins>

> Tells the VAE to train on the x_train dataset for 30 epochs, processing 128 images at a time, and evaluating its progress on the x_test dataset after each epoch (i.e. None for no correspondng validating data values).

**Function**
1. `vae.fit( ... )`
   - Starts the training process for the VAE model.
   - Takes the training data
   - Adjusts the model's parameters (weights and biases) iteratively,
   - Evaluates its performance.
**Parameters**
2. `x_train`: raining dataset containing the input images.
3. `epochs=30`: number of times the VAE will iterate over the entire training dataset.
4. `batch_size=128`: This determines the number of images the VAE processes at once.
5. `validation_data=(x_test, None)`: This provides a separate dataset.
   - `x_test`, likely the MNIST test set
   - `TARGET_VALUES=None`: Are no corresponding target values for the validation data. VAE is a generative model and doesn't require explicit targets for evaluation.

In [ ]:
# Constants
TRAINING_EPOCHS = 30                      # Nos of Training Iterations
TRAINING_BACTCH_SIZE = 128                # No of Images per batch
TARGET_VALUES = None                      # No corresponding target values

In [ ]:
# Train the model
vae.fit(
        x_train,                          # Training Input Dataset
        epochs=TRAINING_EPOCHS,           # Nos of Training Iterations
        batch_size=TRAINING_BACTCH_SIZE,  # No of Images per batch
        validation_data=(                 # Validation data:
                x_test,                     # Separate Test Set
                TARGET_VALUES))             # No corresponding target values



> <hr>

# 9. <ins>Display the generated images - function</ins>

1.  Display a `n*n` 2D manifold of digits
2.  Transform linearly spaced coordinates
    - Linearly spaced coordinates on the unit squares
        - Transformed through the inverse CDF (ppf) of the Gaussian
        - To produce values of the latent variables `z`,
        - Since the prior of the latent space is Gaussian
3.  Generates Image Grid
    - Evenly spacing of a grid
    - Places the reshaped images in the grid
4.  Initialise the Figure creation
5.  Set the pixel boundaries
6.  Initialise the plots features
7.  Display the decode images

In [ ]:
# Display Constants
GRID_SIZE = 30
FIGURE_SIZE = 15
MINIST_STD_SIZE = 28
SCALE_FACTOR = 1.0


### 9.1 *Auxillary Functions*

**Functions**
*   Create Figure: Initialing for plotting
*   Calculate Pixel Ranges
*   PLot axes: Samples, Axes, Ticks, Labels,
*   Display Decoded Images:
    - `plt.imshow(figure, cmap=cmap)`: displays the image data stored in the figure array.
    - `cmap='Greys_r`' sets a colormap to `"Greys_r`": a reversed grayscale colormap,
    - `plt.show()`: renders the plot, making the image grid visible.


In [ ]:
def create_figure(figsize):
    # Inits figure for plotting
    return plt.figure(figsize=(figsize, figsize))

def calculate_pixel_range(digit_size, n, centering=2):
    # Start
    start_range = digit_size // centering
    # End
    end_range = n * digit_size + start_range
    # Pixel Size
    return np.arange(start_range,
                     end_range,
                     digit_size)

def plot_axes(grid_x,
             grid_y,
             pixel_range,
             label_x="z[0]",
             label_y="z[1]",
             round_factor=1):
    # Samples
    sample_range_x = np.round(grid_x, round_factor)
    sample_range_y = np.round(grid_y, round_factor)

    # Axes Ticks, Labels
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    # Labels
    plt.xlabel(label_x)
    plt.ylabel(label_y)

def display_decoded_images(figure, cmap='Greys_r'):
    plt.imshow(figure, cmap=cmap)
    plt.show()

### 9.2 *Display Generated Images*

In [ ]:
# Display generated images
def plot_latent_space(decoder,
                      n=GRID_SIZE,
                      figsize=FIGURE_SIZE):

    # Display a n*n 2D manifold of digits
    digit_size = MINIST_STD_SIZE
    scale = SCALE_FACTOR
    figure = np.zeros((digit_size * n,
                       digit_size * n))


    def generate_image_grid(grid_scale, grid_size):

        # Private Constants
        _REVERSE_SLICE = -1
        _FIRST_ELEMENT = 0
        _FIG_INCREMENT = 1

        # Private Labels - Original Syntax
        ROW_INDEX = 'i'
        Y_COORDINATE = 'yi'
        COLUMN_INDEX = 'j'
        X_COORDINATE = 'xi'
        LATENT_POINT = 'z_sample'
        DECODED_IMAGE = 'x_decoded'
        RESHAPED_DIGIT = 'digit'
        DIGITAL_SIZE = 'digit_size'

        grid_x = np.linspace(-grid_scale, grid_scale, size)
        grid_y = np.linspace(-grid_scale, grid_scale, size)[::_REVERSE_SLICE]

        for ROW_INDEX, Y_COORDINATE in enumerate(grid_y):
            for COLUMN_INDEX, X_COORDINATE in enumerate(grid_x):
                # Latent Point
                LATENT_POINT = np.array([[X_COORDINATE, Y_COORDINATE]])
                # Decoded Image
                DECODED_IMAGE = decoder.predict(LATENT_POINT)
                # Reshaped Image
                RESHAPED_DIGIT = DECODED_IMAGE[_FIRST_ELEMENT].\
                                    reshape(DIGITAL_SIZE, DIGITAL_SIZE)
                # Rows & Columns
                start_row = ROW_INDEX * DIGIT_SIZE
                end_row = (ROW_INDEX + _FIG_INCREMENT) * DIGITAL_SIZE
                start_col = COLUMN_INDEX * DIGITAL_SIZE
                end_col = (COLUMN_INDEX + _FIG_INCREMENT) * DIGITAL_SIZE

                # Figure's reshaped Digit
                figure[start_row:end_row, start_col:end_col] = RESHAPED_DIGIT


    # Call the inner function
    generate_image_grid(scale, n)
    # Figure Creation
    figure = create_figure(figsize)
    # Pixel Boundaries
    pixel_range = calculate_pixel_range(digit_size, n)
    # Axes
    plot_axes(grid_x, grid_y, pixel_range)
    # Display Images
    display_decoded_images(figure)


> <hr>

## 10. <ins>Run the Model</ins>

1. Polt Latent Space:
   - Displays the decoded images.
        - Model: Decoder

In [ ]:
# run the model
plot_latent_space(decoder)

<br>

---
> <center> ~ # ~ </center>
---



## Author

[![LinkedIn](https://img.shields.io/badge/Author-Charles%20J%20Fowler-0077B5?logo=gmail&logoColor=white)](mailto:ipoetdev-github-no-reply@outlook.com "Contact CJ on GItHub email: ipoetdev-github-no-reply@outlook.com") <sup>|</sup> [![LinkedIn](https://img.shields.io/badge/Charles%20J%20Fowler-LinkedIn-0077B5?logo=linkedin&logoColor=white)](https://ie.linkedin.com/in/charlesjfowler "@CharlesJFowler @Linkedin.com") <sup>|</sup> [![LinkedIn](https://img.shields.io/badge/iPoetDev-GitHub-0077B5?logo=GitHub&logoColor=white)](https://github.com/ipoetdev "@iPoetDev @GitHub")

## ChangeLog

| Date<sup>1</sup> | Version | Changed By | Change | Activity | From |
| :--- | :--- | :--- | :--- | :--- | :--- |
| 2024-07-16  | 0.1 | Charles J Fowler  | Source uploaded | Uploaded  | [Source Notebook]( https://colab.research.google.com/drive/1eD7pRKmhVFl0nfwzsoIy9RTtoPMVkZPW?usp=sharing "Author: Marty Bradly") |
| 2024-07-26  | 0.2 | Charles J Fowler  | Draft Portfolio version | Modify  | --- |  
<sup>1</sup>: `YYYY-MM-DD